In [1]:
import os

import theano
import theano.tensor as T
from theano.tensor.shared_randomstreams import RandomStreams
from lasagne import init
from lasagne.layers import (Layer, InputLayer, MergeLayer, DenseLayer, 
                            DimshuffleLayer, ElemwiseSumLayer, 
                            ReshapeLayer, NonlinearityLayer,
                            Conv2DLayer, MaxPool2DLayer, Upscale2DLayer,
                            get_all_params, get_output)
from lasagne.objectives import categorical_crossentropy
from lasagne.nonlinearities import rectify, sigmoid, softmax
from lasagne.updates import total_norm_constraint, adam
from parmesan.layers import SampleLayer
from parmesan.distributions import log_normal
from models.adgmssl import ADGMSSL


DEBUG: nvcc STDOUT mod.cu
   Creating library D:/temp/theano/compiledir_Windows-10-10.0.10586-Intel64_Family_6_Model_58_Stepping_9_GenuineIntel-2.7.11-64/tmpnporar/265abc51f7c376c224983485238ff1a5.lib and object D:/temp/theano/compiledir_Windows-10-10.0.10586-Intel64_Family_6_Model_58_Stepping_9_GenuineIntel-2.7.11-64/tmpnporar/265abc51f7c376c224983485238ff1a5.exp

Using gpu device 0: GeForce GTX 970 (CNMeM is disabled, CuDNN 4004)
d:\packages\theano\theano\tensor\signal\downsample.py:5: UserWarning: downsample module has been moved to the pool module.
  warnings.warn("downsample module has been moved to the pool module.")


EVIL HACK: Disable cuDNN check


In [2]:
from data_preparation import mnist
import numpy as np

In [3]:
"""
Train a auxiliary deep generative model on the mnist dataset with 100 evenly distributed labels.
"""
n_labeled = 100  # The total number of labeled data points.
n_samples = 100  # The number of sampled labeled data points for each batch.
n_batches = 100  # The number of batches.
mnist_data = mnist.load_semi_supervised(n_batches=n_batches, n_labeled=n_labeled, n_samples=n_samples,
                                        filter_std=0.0, seed=123456, train_valid_combine=True)

mnist_data = ((mnist_data[0][0].reshape((-1, 1, 28, 28)), mnist_data[0][1]),
              (mnist_data[1][0].reshape((-1, 1, 28, 28)), mnist_data[1][1]),
              (mnist_data[2][0].reshape((-1, 1, 28, 28)), mnist_data[2][1]))

shape_x = (1,28,28)
n = mnist_data[0][0].shape[0]  # Datapoints in the dataset, input features.
bs = n / n_batches  # The batchsize.

def f_enc(layer):
    layer = Conv2DLayer(layer, num_filters=32, filter_size=(5,5), W=init.HeUniform('relu'), nonlinearity=rectify)
    layer = MaxPool2DLayer(layer, (2,2))
    layer = Conv2DLayer(layer, num_filters=32, filter_size=(3,3), W=init.HeUniform('relu'), nonlinearity=rectify)
    layer = Conv2DLayer(layer, num_filters=32, filter_size=(3,3), W=init.HeUniform('relu'), nonlinearity=rectify)
    layer = MaxPool2DLayer(layer, (2,2))
    return layer

def f_dec(layer):
    layer = ReshapeLayer(layer, ([0], 32, 4, 4))
    layer = Upscale2DLayer(layer, (2,2))
    layer = Conv2DLayer(layer, num_filters=32, filter_size=(3,3), W=init.HeUniform('relu'), nonlinearity=rectify, pad='full')
    layer = Conv2DLayer(layer, num_filters=32, filter_size=(3,3), W=init.HeUniform('relu'), nonlinearity=rectify, pad='full')
    layer = Upscale2DLayer(layer, (2,2))
    layer = Conv2DLayer(layer, num_filters=1, filter_size=(5,5), W=init.HeUniform('relu'), nonlinearity=rectify, pad='full')
    return layer

# Initialize the auxiliary deep generative model.
model = ADGMSSL(shape_x=shape_x, n_a=100, n_z=100, n_y=10, n_xhat=512, f_enc=f_enc, f_dec=f_dec,
                z_hidden=[500], y_hidden=[500],
                trans_func=rectify, x_dist='bernoulli')

# Get the training functions.
f_train, f_test, f_validate, train_args, test_args, validate_args = model.build_model(*mnist_data)

DEBUG: nvcc STDOUT mod.cu
   Creating library D:/temp/theano/compiledir_Windows-10-10.0.10586-Intel64_Family_6_Model_58_Stepping_9_GenuineIntel-2.7.11-64/tmpx1a_hi/f44924add8b3f979f00f17b1cf27bcbd.lib and object D:/temp/theano/compiledir_Windows-10-10.0.10586-Intel64_Family_6_Model_58_Stepping_9_GenuineIntel-2.7.11-64/tmpx1a_hi/f44924add8b3f979f00f17b1cf27bcbd.exp

DEBUG: nvcc STDOUT mod.cu
   Creating library D:/temp/theano/compiledir_Windows-10-10.0.10586-Intel64_Family_6_Model_58_Stepping_9_GenuineIntel-2.7.11-64/tmpfv_arx/71ea070f78581fad5854acbbcc8f4ad8.lib and object D:/temp/theano/compiledir_Windows-10-10.0.10586-Intel64_Family_6_Model_58_Stepping_9_GenuineIntel-2.7.11-64/tmpfv_arx/71ea070f78581fad5854acbbcc8f4ad8.exp

DEBUG: nvcc STDOUT mod.cu
   Creating library D:/temp/theano/compiledir_Windows-10-10.0.10586-Intel64_Family_6_Model_58_Stepping_9_GenuineIntel-2.7.11-64/tmpoyzc8e/981d5cd806d6da3a11b6318ba71556b6.lib and object D:/temp/theano/compiledir_Windows-10-10.0.10586-Inte

### BUILDING MODEL ###


 mod.cu
   Creating library D:/temp/theano/compiledir_Windows-10-10.0.10586-Intel64_Family_6_Model_58_Stepping_9_GenuineIntel-2.7.11-64/tmpnxqpw3/aa55c7530a1d1eab1bfb740f74ec52f1.lib and object D:/temp/theano/compiledir_Windows-10-10.0.10586-Intel64_Family_6_Model_58_Stepping_9_GenuineIntel-2.7.11-64/tmpnxqpw3/aa55c7530a1d1eab1bfb740f74ec52f1.exp



In [ ]:
import time
import cPickle as pkl
import os
path = "output/notebook"
if not os.path.exists(path):
    os.mkdir(path)

bs_l = n_samples
beta = 1200.
lr = 3e-4
beta1 = 0.9
beta2 = 0.999
samples = 1
epochs = 200
for epoch in range(epochs):
    train_outputs = []
    start = time.time()
    for i in xrange(n_batches):
        train_output = f_train(i, bs, bs_l, beta, lr, beta1, beta2, samples)
        train_outputs.append(train_output)
    lb = np.mean(np.array(train_outputs), axis=0)
    end = time.time() - start
    
    test_outputs = f_test(samples)
    class_err_test = np.mean(np.array(test_outputs), axis=0)
    
    # model_params = [param.get_value() for param in model]
    # pkl.dump(model_params, open(os.path.join(path, "model.pkl"), "wb"))
    
    print "[epoch,time,train[lb],test[err]];%i;%0.2f;%0.4f;%0.2f%%" % (epoch+1, end, lb, class_err_test)